We merge the existing Dataframe with market data to extract CryptoCompare's social media channel...

In [1]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
%load_ext autoreload
%matplotlib inline

In [2]:
sys.path.append(os.path.abspath(os.path.join('..')))
from utils import toUnixTimestamp

In [3]:
pump_data = pd.read_pickle('pump_data.pkl')

__REMARKS__: 

We only need to recycly the timestamp variabels and use them to merge for social data

In [4]:
pump_data.head()

,channelLink,channelTitle,currency,duration,exchange,priceBeforePump,signalTime,pumptime_edited,timestamp,pump_weekday,...,high_day_7,low_day_7,close_day_7,volumeto_day_7,pump_hour_7,open_7,high_7,low_7,close_7,volumeto_7
2,https://t.me/bitcoinpumpgroup,Bitcoin Pump Group,WAN,02:12,Binance,0.000034,2019-08-30T16:41:30Z,2019-08-30 16:41:30+00:00,1567183290,4,...,0.000041,0.000024,0.000038,1999.64,1.567159e+09,0.000031,0.000032,0.000031,0.000031,9.8470
3,https://t.me/bitcoinprofitcoach,Bitcoin Profit Coach,QSP,10:07,Binance,0.000001,2019-08-29T19:46:43Z,2019-08-29 19:46:43+00:00,1567108003,3,...,0.000001,0.000001,0.000001,101.24,1.567084e+09,0.000001,0.000001,0.000001,0.000001,0.3351
4,https://t.me/rocketpumptrader,Global Pump Signals 🚀,QSP,01:14,Binance,0.000001,2019-08-29T18:00:16Z,2019-08-29 18:00:16+00:00,1567101616,3,...,0.000001,0.000001,0.000001,101.24,1.567076e+09,0.000001,0.000001,0.000001,0.000001,0.8835
5,https://t.me/palmvenicebeach,Palm Venice Beach,QSP,00:19,Binance,0.000002,2019-07-07T15:17:42Z,2019-07-07 15:17:42+00:00,1562512662,6,...,0.000002,0.000002,0.000002,13.52,1.562486e+09,0.000002,0.000002,0.000002,0.000002,1.1320
6,https://t.me/BinanceMegaPump1,Binance Mega Pump,QSP,00:46,Binance,0.000002,2019-07-05T15:00:01,2019-07-05 15:00:01+00:00,1562338801,4,...,0.000002,0.000002,0.000002,93.44,1.562314e+09,0.000002,0.000002,0.000002,0.000002,0.1722


In [5]:
def getSocialfor_X_Hours(currency:str, coin_id: str, pump:pd.DataFrame, h_back: int = 7):
    pump_df = pump.copy()
    try:
        filename = '../Social_Data/Hourly_'+currency+'_'+coin_id+'.pkl'
        print("---Looking for ",filename)
        social = pd.read_pickle(filename)
        #Option 1: Choose only some
#         base_append = ['analysis_page_views', 'charts_page_views',
#                        'markets_page_views', 'overview_page_views',
#                         'total_page_views', 'trades_page_views'
        base_append  = ['analysis_page_views', 'charts_page_views', 'code_repo_closed_issues',
       'code_repo_closed_pull_issues', 'code_repo_forks',
       'code_repo_open_issues', 'code_repo_open_pull_issues',
       'code_repo_stars', 'code_repo_subscribers', 'comments', 'fb_likes',
       'fb_talking_about', 'followers', 'forum_page_views',
       'influence_page_views', 'markets_page_views', 'overview_page_views',
       'points', 'posts', 'reddit_active_users', 'reddit_comments_per_day',
       'reddit_comments_per_hour', 'reddit_posts_per_day',
       'reddit_posts_per_hour', 'reddit_subscribers', 'time',
       'total_page_views', 'trades_page_views', 'twitter_favourites',
       'twitter_followers', 'twitter_following', 'twitter_lists',
       'twitter_statuses', 'date', 'date_utc']
        back_list = []
        
        for h in range(h_back+1): 
            hour_var = 'pump_hour_'+str(h)
            back_list.append(hour_var)
            
            #Make a copy of the ohlcv with new variable names
            append_name = ['time'] + [name+'_'+str(h) for name in base_append]
            social_h = social[['time']+base_append]
            social_h.columns = append_name
            #print(ohlcv_h.columns.values)
           
            #Perform the merge to get data
            pump = pd.merge(left=pump_df,left_on=hour_var,right=social_h,right_on='time',how='left')
            pump = pump.drop('time',axis=1)
            pump_df = pump.reset_index(drop=True)
            
        return pump_df
    except FileNotFoundError:
        print('No HOURLY SOCIAL data found on coin!')
        return {'currency':currency,'id': coin_id}

## VERIFY THE FORMAT of SOCIAL MEDIA Data

In [6]:
adx  = pd.read_pickle('../Social_Data/Hourly_ADX_170452.pkl')

In [7]:
adx.head(2)

,analysis_page_views,charts_page_views,code_repo_closed_issues,code_repo_closed_pull_issues,code_repo_forks,code_repo_open_issues,code_repo_open_pull_issues,code_repo_stars,code_repo_subscribers,comments,...,time,total_page_views,trades_page_views,twitter_favourites,twitter_followers,twitter_following,twitter_lists,twitter_statuses,date,date_utc
0,0,0,0,0,0,0,0,0,0,0,...,1531209600,0,0,0,0,0,0,0,2018-07-10 01:00:00,2018-07-10 08:00:00+00:00
1,0,0,0,0,0,0,0,0,0,0,...,1531213200,0,0,0,0,0,0,0,2018-07-10 02:00:00,2018-07-10 09:00:00+00:00


In [8]:
adx.date_utc.describe()

count                         10005
unique                        10000
top       2019-06-08 16:00:00+00:00
freq                              2
first     2018-07-10 08:00:00+00:00
last      2019-08-31 00:00:00+00:00
Name: date_utc, dtype: object

...Observe that the time variable is already in date_utc. So no need to convert!

In [9]:
adx['comments'].head()

0    0
1    0
2    0
3    0
4    0
Name: comments, dtype: int64

In [10]:
adx[(adx.time >= 1564142400) & (adx.time <= 1564164000 )][['time','analysis_page_views', 'charts_page_views',
                       'markets_page_views', 'overview_page_views',
                        'total_page_views', 'trades_page_views']]

,time,analysis_page_views,charts_page_views,markets_page_views,overview_page_views,total_page_views,trades_page_views
1148,1564142400,1748,10440,4075,79140,106470,1648
1149,1564146000,1748,10440,4075,79140,106470,1648
1150,1564149600,1748,10440,4075,79140,106470,1648
1151,1564153200,1748,10440,4075,79140,106470,1648
1152,1564156800,1748,10440,4075,79140,106470,1648
1153,1564160400,1748,10440,4075,79140,106470,1648
1154,1564164000,1748,10440,4075,79140,106470,1648


## Test on ADX Coin

In [11]:
p = pump_data[pump_data.currency=='ADX']
p.filter(regex='pump_hour')

,pump_hour,pump_hour_0,pump_hour_1,pump_hour_2,pump_hour_3,pump_hour_4,pump_hour_5,pump_hour_6,pump_hour_7
107,17,1564164000,1564160400,1564156800,1564153200,1564149600,1564146000,1564142400,1.564139e+09
108,17,1562346000,1562342400,1562338800,1562335200,1562331600,1562328000,1562324400,1.562321e+09
109,19,1556218800,1556215200,1556211600,1556208000,1556204400,1556200800,1556197200,1.556194e+09


In [12]:
merge_data = getSocialfor_X_Hours('ADX','170452',p)

---Looking for  ../Social_Data/Hourly_ADX_170452.pkl


In [13]:
merge_data.head()

,channelLink,channelTitle,currency,duration,exchange,priceBeforePump,signalTime,pumptime_edited,timestamp,pump_weekday,...,time_7,total_page_views_7,trades_page_views_7,twitter_favourites_7,twitter_followers_7,twitter_following_7,twitter_lists_7,twitter_statuses_7,date_7,date_utc_7
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,00:00,Binance,0.000011,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999,4,...,1564138800,106470,1648,1473,54064,254,797,691,2019-07-26 04:00:00,2019-07-26 11:00:00+00:00
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,00:53,Binance,0.000012,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032,4,...,1562320800,106321,1648,1470,54168,254,795,686,2019-07-05 03:00:00,2019-07-05 10:00:00+00:00
2,https://t.me/MaximumPump,Maximum Pump,ADX,00:02,Binance,0.000029,2019-04-25T19:15:21,2019-04-25 19:15:21+00:00,1556219721,3,...,1556193600,105926,1646,1456,54782,253,792,660,2019-04-25 05:00:00,2019-04-25 12:00:00+00:00


In [14]:
adx.columns

Index(['analysis_page_views', 'charts_page_views', 'code_repo_closed_issues',
       'code_repo_closed_pull_issues', 'code_repo_forks',
       'code_repo_open_issues', 'code_repo_open_pull_issues',
       'code_repo_stars', 'code_repo_subscribers', 'comments', 'fb_likes',
       'fb_talking_about', 'followers', 'forum_page_views',
       'influence_page_views', 'markets_page_views', 'overview_page_views',
       'points', 'posts', 'reddit_active_users', 'reddit_comments_per_day',
       'reddit_comments_per_hour', 'reddit_posts_per_day',
       'reddit_posts_per_hour', 'reddit_subscribers', 'time',
       'total_page_views', 'trades_page_views', 'twitter_favourites',
       'twitter_followers', 'twitter_following', 'twitter_lists',
       'twitter_statuses', 'date', 'date_utc'],
      dtype='object')

In [15]:
adx.filter(regex='._views').columns

Index(['analysis_page_views', 'charts_page_views', 'forum_page_views',
       'influence_page_views', 'markets_page_views', 'overview_page_views',
       'total_page_views', 'trades_page_views'],
      dtype='object')

__OBSERVE__: We see that a lot of the information is very repetitive. Consider some form of aggregation to shorten them all.

## Perform Mergeing Session

In [16]:
coin_list = pd.read_pickle('../Coin_Data/coin_list.pkl')
coin_list = coin_list[['currency','Id']].drop_duplicates()
#Only keep the rows where ID is available 
ind = coin_list.Id[coin_list.Id.isna()].index
coin_list = coin_list.drop(index=ind)

In [17]:
coin_list.head()

,currency,Id
0,KMD,26132
1,WAN,240142
2,QSP,397757
3,GVT,385952
4,POWR,339617


In [18]:
p = pd.read_pickle('pump_data.pkl')
p.head()

,channelLink,channelTitle,currency,duration,exchange,priceBeforePump,signalTime,pumptime_edited,timestamp,pump_weekday,...,high_day_7,low_day_7,close_day_7,volumeto_day_7,pump_hour_7,open_7,high_7,low_7,close_7,volumeto_7
2,https://t.me/bitcoinpumpgroup,Bitcoin Pump Group,WAN,02:12,Binance,0.000034,2019-08-30T16:41:30Z,2019-08-30 16:41:30+00:00,1567183290,4,...,0.000041,0.000024,0.000038,1999.64,1.567159e+09,0.000031,0.000032,0.000031,0.000031,9.8470
3,https://t.me/bitcoinprofitcoach,Bitcoin Profit Coach,QSP,10:07,Binance,0.000001,2019-08-29T19:46:43Z,2019-08-29 19:46:43+00:00,1567108003,3,...,0.000001,0.000001,0.000001,101.24,1.567084e+09,0.000001,0.000001,0.000001,0.000001,0.3351
4,https://t.me/rocketpumptrader,Global Pump Signals 🚀,QSP,01:14,Binance,0.000001,2019-08-29T18:00:16Z,2019-08-29 18:00:16+00:00,1567101616,3,...,0.000001,0.000001,0.000001,101.24,1.567076e+09,0.000001,0.000001,0.000001,0.000001,0.8835
5,https://t.me/palmvenicebeach,Palm Venice Beach,QSP,00:19,Binance,0.000002,2019-07-07T15:17:42Z,2019-07-07 15:17:42+00:00,1562512662,6,...,0.000002,0.000002,0.000002,13.52,1.562486e+09,0.000002,0.000002,0.000002,0.000002,1.1320
6,https://t.me/BinanceMegaPump1,Binance Mega Pump,QSP,00:46,Binance,0.000002,2019-07-05T15:00:01,2019-07-05 15:00:01+00:00,1562338801,4,...,0.000002,0.000002,0.000002,93.44,1.562314e+09,0.000002,0.000002,0.000002,0.000002,0.1722


In [19]:
coin_social_hour = []
for index,row in coin_list.iterrows():
    currency = row['currency']
    coin_id  = row['Id']
    temp = None
    #Must select only the rows with the right currency 
    pump = p.loc[p.currency == currency]
    temp = getSocialfor_X_Hours(currency,coin_id,pump,7)
    if isinstance(temp, pd.DataFrame) : 
        merge_data = pd.concat([merge_data,temp],axis=0,sort=False)
    else:
        #Append social data only when the coin is available
        coins_social_hour.append(temp)
           

---Looking for  ../Social_Data/Hourly_KMD_26132.pkl
---Looking for  ../Social_Data/Hourly_WAN_240142.pkl
---Looking for  ../Social_Data/Hourly_QSP_397757.pkl
---Looking for  ../Social_Data/Hourly_GVT_385952.pkl
---Looking for  ../Social_Data/Hourly_POWR_339617.pkl
---Looking for  ../Social_Data/Hourly_OST_369132.pkl
---Looking for  ../Social_Data/Hourly_SNT_137013.pkl
---Looking for  ../Social_Data/Hourly_BTS_5039.pkl
---Looking for  ../Social_Data/Hourly_VIB_198710.pkl
---Looking for  ../Social_Data/Hourly_QLC_637681.pkl
---Looking for  ../Social_Data/Hourly_ONG_428300.pkl
---Looking for  ../Social_Data/Hourly_GXS_659770.pkl
---Looking for  ../Social_Data/Hourly_TFD_784553.pkl
---Looking for  ../Social_Data/Hourly_CDT_177139.pkl
---Looking for  ../Social_Data/Hourly_DLT_220204.pkl
---Looking for  ../Social_Data/Hourly_VIA_5015.pkl
---Looking for  ../Social_Data/Hourly_DATA_369151.pkl
---Looking for  ../Social_Data/Hourly_FUN_178978.pkl
---Looking for  ../Social_Data/Hourly_ALGO_930992

---Looking for  ../Social_Data/Hourly_GNO_66193.pkl
---Looking for  ../Social_Data/Hourly_FIRE_29196.pkl
---Looking for  ../Social_Data/Hourly_XPRO_38380.pkl
---Looking for  ../Social_Data/Hourly_XMG_5331.pkl
---Looking for  ../Social_Data/Hourly_TRX_310829.pkl
---Looking for  ../Social_Data/Hourly_LTC_3808.pkl
---Looking for  ../Social_Data/Hourly_IQN_911125.pkl
---Looking for  ../Social_Data/Hourly_SEL_41967.pkl
---Looking for  ../Social_Data/Hourly_ICN_25921.pkl
---Looking for  ../Social_Data/Hourly_UNITS_38032.pkl
---Looking for  ../Social_Data/Hourly_USDT_171986.pkl
---Looking for  ../Social_Data/Hourly_AE_190978.pkl
---Looking for  ../Social_Data/Hourly_MNZ_341059.pkl
---Looking for  ../Social_Data/Hourly_ETC_5324.pkl
---Looking for  ../Social_Data/Hourly_HAC_321347.pkl
---Looking for  ../Social_Data/Hourly_PRO_185695.pkl
---Looking for  ../Social_Data/Hourly_PAI_903863.pkl
---Looking for  ../Social_Data/Hourly_HDG_237407.pkl
---Looking for  ../Social_Data/Hourly_CL_659393.pkl
--

In [20]:
#All the coin should be available...
coin_social_hour

[]

In [21]:
merge_data = merge_data.drop_duplicates().reset_index(drop=True)

In [22]:
merge_data.shape

(426, 383)

## VERIFY that the Number of Currency and Exchanges stay consistent with Originally

In [23]:
merge_data['currency'].drop_duplicates().count()

154

In [24]:
merge_data[['currency','exchange']].drop_duplicates().count()

currency    179
exchange    179
dtype: int64

In [25]:
pump_data[['currency','exchange']].drop_duplicates().count()

currency    179
exchange    179
dtype: int64

In [26]:
merge_data.head()

,channelLink,channelTitle,currency,duration,exchange,priceBeforePump,signalTime,pumptime_edited,timestamp,pump_weekday,...,time_7,total_page_views_7,trades_page_views_7,twitter_favourites_7,twitter_followers_7,twitter_following_7,twitter_lists_7,twitter_statuses_7,date_7,date_utc_7
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,00:00,Binance,0.000011,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999,4,...,1564138800,106470,1648,1473,54064,254,797,691,2019-07-26 04:00:00,2019-07-26 11:00:00+00:00
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,00:53,Binance,0.000012,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032,4,...,1562320800,106321,1648,1470,54168,254,795,686,2019-07-05 03:00:00,2019-07-05 10:00:00+00:00
2,https://t.me/MaximumPump,Maximum Pump,ADX,00:02,Binance,0.000029,2019-04-25T19:15:21,2019-04-25 19:15:21+00:00,1556219721,3,...,1556193600,105926,1646,1456,54782,253,792,660,2019-04-25 05:00:00,2019-04-25 12:00:00+00:00
3,https://t.me/bitcoinpumpgroup,Bitcoin Pump Group,WAN,02:12,Binance,0.000034,2019-08-30T16:41:30Z,2019-08-30 16:41:30+00:00,1567183290,4,...,1567159200,133755,685,0,0,0,0,0,2019-08-30 03:00:00,2019-08-30 10:00:00+00:00
4,https://t.me/bitcoinprofitcoach,Bitcoin Profit Coach,QSP,10:07,Binance,0.000001,2019-08-29T19:46:43Z,2019-08-29 19:46:43+00:00,1567108003,3,...,1567083600,67905,1399,1793,56509,458,739,1495,2019-08-29 06:00:00,2019-08-29 13:00:00+00:00


In [27]:
merge_data.columns.values

array(['channelLink', 'channelTitle', 'currency', 'duration', 'exchange',
       'priceBeforePump', 'signalTime', 'pumptime_edited', 'timestamp',
       'pump_weekday', 'pump_date', 'pump_hour', 'pump_minute',
       'pump_hour_0', 'open_0', 'high_0', 'low_0', 'close_0',
       'volumeto_0', 'pump_hour_1', 'open_1', 'high_1', 'low_1',
       'close_1', 'volumeto_1', 'pump_hour_2', 'open_2', 'high_2',
       'low_2', 'close_2', 'volumeto_2', 'pump_hour_3', 'open_3',
       'high_3', 'low_3', 'close_3', 'volumeto_3', 'pump_hour_4',
       'open_4', 'high_4', 'low_4', 'close_4', 'volumeto_4',
       'pump_hour_5', 'open_5', 'high_5', 'low_5', 'close_5',
       'volumeto_5', 'pump_hour_6', 'open_6', 'high_6', 'low_6',
       'close_6', 'volumeto_6', 'pump_day_1', 'open_day_1', 'high_day_1',
       'low_day_1', 'close_day_1', 'volumeto_day_1', 'pump_day_2',
       'open_day_2', 'high_day_2', 'low_day_2', 'close_day_2',
       'volumeto_day_2', 'pump_day_3', 'open_day_3', 'high_day_3',
     

In [28]:
merge_data.to_pickle('pump_data_with_social.pkl')

 *------END OF CODE----*